#Students: Elisei Smirnov, Amine Trabelsi, Liana Mardanova
## Group: DS-01
## Task description: [Link](https://www.kaggle.com/competitions/santa-workshop-tour-2019/data)



In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import copy
from typing import List, Tuple, Dict, Any

# Read the family data into a DataFrame, using 'family_id' as the index
data_file_path = 'family_data.csv'
data = pd.read_csv(data_file_path, index_col='family_id')

# Read the sample submission data into a DataFrame, using 'family_id' as the index
sample_submission_file_path = 'sample_submission.csv'
sample_submission = pd.read_csv(sample_submission_file_path, index_col='family_id')


### Given information

In [ ]:
data.head()

In [ ]:
sample_submission.head()

### Penalty cost for Santa

> The total number of people attending the workshop each day must be between $125$ - $300$; if even one day is outside these occupancy constraints, the submission will error and will not be scored.

> Santa provides consolation gifts to families. These sum up per family, and the total represents the **$preference:cost$**:
- `choice_0`: no consolation gifts
- `choice_1`: \$50
- `choice_2`: \\$50, \$9 for each family member
- `choice_3`: \\$100, \$9 for each family member
- `choice_4`: \\$200, \$9 for each family member
- `choice_5`: \\$200, \$18 for each family member
- `choice_6`: \\$300, \$18 for each family member
- `choice_7`: \\$300, \$36 for each family member
- `choice_8`: \\$400, \$36 for each family member
- `choice_9`: \\$500, \\$36 for each family member, \\$199 for each family member
- `otherwise`: \\$500, \\$36 for each family member, \$398 for each family member.

> $$accoutning:penalty = \sum_{d=100}^1\frac{N_d-125}{400}N_d^{\frac{1}{2}+\frac{|N_d-N_{d+1}|}{50}}$$
> where $N_d$ is the occupancy of the current day, and $N_{d+1}$ is the occupancy of the previous day (since we're counting backwards from Christmas! For the initial condition of $d=100$, $N_{101}=N_{100}$.

> **Finally $score=preference:cost+accounting:penalty$**

### Genetic Algorithm Implementation

In [ ]:
"""
Some parts of tge code based on already provided by comptetition starter notebook:
https://www.kaggle.com/code/inversion/santa-s-2019-starter-notebook

"""

class GeneticScheduler:
    """
    This class reads family preference data, generates an initial population, and iteratively
    evolves this population to find an optimal schedule that minimizes fitness function. This
    function is based on the costs and constraints, provided in the description of Santa's
    Workshop Tour 2019 problem.

    This class encapsulates the logic for scheduling families based on their preferences
    and constraints using a genetic algorithm. It includes methods for initializing the
    population, performing crossover and mutation operations, selecting parents, and
    evaluating fitness.

    Attributes:
    - file_location (str): File location of the family preference data.
    - answer_location (str): File location to save the best solution.
    - num_days (int): Number of days Santa's workshop is open.
    - family_num (int): Number of families.
    - MIN_PEOPLE (int): Minimum constraint of people in a workshop per day.
    - MAX_PEOPLE (int): Maximum constraint of people in a workshop per day.
    - preferences_num (int): Number of preferences each family provides.
    - population_size (int): Size of the population in each generation.
    - mutation_rate (float): Probability of type of mutation for each chromosome.
    - crossover_rate (float): Probability of crossover between two parents.
    - num_generations (int): Number of generations to run the genetic algorithm.
    - best_solution (numpy.ndarray): Best solution found so far.
    - best_fitness (float): Fitness score of the best solution.
    - population (list): Current population of chromosomes.
    - family_pref_matrix (numpy.ndarray): Matrix of family preferences.
    - family_sizes (numpy.ndarray): Array of family sizes.
    - consolation_gifts (dict): Price Santa pays for each family according to assigned day.
    """
    def __init__(self,
                 file_location: str,
                 answer_location: str,
                 population_size: int = 20,
                 mutation_rate: float = 0.9,
                 num_generations: int = 1000,
                 family_num: int = 5000,
                 number_of_days: int = 100,
                 min_constraint: int = 125,
                 max_constraint: int = 300,
                 number_of_preferences: int = 10,
                 crossover_rate: float = 0.9):
        """
        Initializes the GeneticScheduler.

        Args:
            file_location (str): File location of the family preference data.
            answer_location (str): File location to save the best solution.
            population_size (int): Size of the population in each generation.
            mutation_rate (float): Probability to mutate for each chromosome.
            num_generations (int): Number of generations to run the genetic algorithm.
            family_num (int): Number of families.
            number_of_days (int): Number of days Santa's workshop is open.
            min_constraint (int): Minimum constraint of people in a workshop per day.
            max_constraint (int): Maximum constraint of people in a workshop per day.
            number_of_preferences (int): Number of preferences each family provides.
            crossover_rate (float): Probability to perform crossover for pair of chromosomes.
        """
        self.file_location = file_location
        self.answer_location = answer_location
        self.num_days = number_of_days
        self.family_num = family_num
        self.MIN_PEOPLE = min_constraint
        self.MAX_PEOPLE = max_constraint
        self.preferences_num = number_of_preferences

        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.num_generations = num_generations

        # Read csv file with preferences for each family (provided in the task)
        family_data = pd.read_csv(self.file_location)
        # Retreive column names where the preferences are stored
        pref_columns = [f'choice_{i}' for i in range(10)]

        # Numpy matrix (5000, 10) with each 10 preferences for each family.
        # Note that the size of the matrix can be easily changed for your data.
        # However, the Santa's Workshop Tour 2019 provides exactly 5000 families,
        # with 10 preferences in each.
        self.family_pref_matrix = family_data[pref_columns].to_numpy()

        # Numpy array of family sizes, where index is family_id
        self.family_sizes = family_data['n_people'].to_numpy()

        # Retreive every possible family size in the provided file
        unique_family_sizes = np.sort(np.unique(self.family_sizes))

        # Price Santa pays for each family according to assigned day
        # key: n -- size of the family
        # value: list of costs for every possible day assigment (1st preference, etc.)
        self.consolation_gifts = {
            n: np.array([
                0,
                50,
                50 + 9 * n,
                100 + 9 * n,
                200 + 9 * n,
                200 + 18 * n,
                300 + 18 * n,
                300 + 36 * n,
                400 + 36 * n,
                500 + 36 * n + 199 * n,
                500 + 36 * n + 398 * n
            ])
            for n in unique_family_sizes
        }

        # already during class initialization generate inital population for GA
        self.population = self.generate_initial_population()

        # Store best found solution ever
        self.best_solution = None
        # Store best calculated fintess ever
        self.best_fitness = float('inf')

    def fitness_function(self, chromosome: np.ndarray) -> float:
        """
        Calculates the fitness score of a chromosome.
        - It includes restriction on the total number of people attending Santa;
        - Preference:cost which is consolation gifts for each family;
        - Accounting:penalty which accounts taxes, cleaning costs and etc.
        Args:
            chromosome (numpy.ndarray): Chromosome representing the assignment of families to days.

        Returns:
            float: Fitness score of the chromosome.
        """
        # A list of indices corresponding to the days on which families are
        # scheduled to visit Santa's workshop according to the chromosome.
        assigned_days = [chromosome[i] for i in range(self.family_num)]

        fitness = 0

        # Counter of total people for every possible day reserved for workshop
        day_workshops_number = {d: 0 for d in range(1, self.num_days + 1)}

        # Looping over each family;
        # fam_size - number of people in family
        # day - chosen day for that family in the given chromosome
        # preferences - numpy.ndarray of 10 (for this task) preferences of that family
        for fam_size, day, preferences in zip(self.family_sizes, assigned_days, self.family_pref_matrix):
            # add the number of family members to the daily number of people
            day_workshops_number[day] += fam_size

            if day not in preferences:
                # Calculate the penalty for not assigning a preferred day
                fitness += self.consolation_gifts[fam_size][-1]
            else:
                # Calculate the penalty for assigning a preferred day
                index = np.where(preferences == day)[0][0]  # Get the index of the chosen day in preferences
                fitness += self.consolation_gifts[fam_size][index]

        for n in day_workshops_number.values():
            if (n < self.MIN_PEOPLE) or (n > self.MAX_PEOPLE):
                # soft constraints for better convergence for the fitness to minimum
                # 10**9 is not a magical number, but it were given in the task description
                fitness += 10 ** 9

        # Calculate accounting:penalty (taxes cost, etc.). The formula were given in the task itself
        accounting_penalty = max(0, (day_workshops_number[self.num_days] - 125.0) / 400.0 * day_workshops_number[self.num_days] ** 0.5)
        yesterday_workshops_number = day_workshops_number[self.num_days]
        for day in range(self.num_days - 1, 0, -1):
            today = day_workshops_number[day]
            diff = abs(today - yesterday_workshops_number)
            accounting_penalty += max(0, (today - 125.0) / 400.0 * today ** (0.5 + diff / 50.0))
            yesterday_workshops_number = today

        fitness += accounting_penalty
        return fitness

    def generate_initial_population(self) -> List[np.ndarray]:
        """
        Generates the initial population of chromosomes.

        Returns:
            list: Initial population of chromosomes.
        """
        population = []
        for i in range(self.population_size):
            # We are firstly initialize chromosome with zeros
            # Later each value will represent chosen day for ith family
            # where i is index in the chromosome numpy array
            chromosome = np.zeros(self.family_num, dtype=int)

            # We'll use intelligent chromosome creation that will try to
            # evenly distribute each family by day
            days_capacity = {d: 0 for d in range(1, self.num_days + 1)}
            # get families indexes as a list
            families_order = list(range(self.family_num))
            # shuffle this list of family indexes to randomly look at
            # families that are not in the any predefined order
            random.shuffle(families_order)

            for i in families_order:
                if chromosome[i] == 0:
                    # if chromosome didn't store any day choice yet for that
                    # family
                    flag = True
                    for j in range(self.preferences_num):
                        # trying to go through every preference of that family
                        day_candidate = self.family_pref_matrix[i][j]
                        day_load = days_capacity[day_candidate]
                        if day_load + self.family_sizes[i] < (self.MIN_PEOPLE + self.MAX_PEOPLE) / 2:
                            # and make sure the day isn't filled with too many families yet.
                            # if not, then chose that day and store it in the chromosome
                            chromosome[i] = day_candidate
                            days_capacity[day_candidate] += self.family_sizes[i]
                            flag = False
                            # break in order not to consider any other day from preferences
                            break
                    if flag:
                        # if all days from preferences are loaded to much, then just assign
                        # random day as a choice for that family in the chromosome
                        chromosome[i] = np.random.randint(1, self.num_days + 1)
            # append chromosome to the population
            population.append(chromosome)

        return population

    def mutation(self, chromosome: np.ndarray) -> np.ndarray:
        """
        Mutates a chromosome based on mutation rate.

        This method introduces random changes into the chromosome to explore new solutions with probability = self.mutation_rate.
        It does so by randomly selecting a number of genes to mutate. For each gene selected,
        it either swaps the gene's value with one of its preferences or assigns it a random day (type of mutation also based on self.mutation_rate).
        This process helps in exploring the solution space more thoroughly and avoiding premature convergence
        to suboptimal solutions.

        Args:
            chromosome (numpy.ndarray): Chromosome to be mutated.

        Returns:
            numpy.ndarray: Mutated chromosome.
        """
        # Decide whether to mutate the gene based on the mutation rate.
        if random.random() < self.mutation_rate:
            # Determine the number of mutations to apply based on a random number.
            # This introduces variability in the mutation process.
            num_mutations = random.randint(1, 5)

            # Iterate over the number of mutations to apply.
            for _ in range(num_mutations):
                # Select a random gene (family) to mutate.
                gene_index = random.randint(0, len(chromosome) - 1)

                # Decide which type of mutation to apply based on the mutation rate.
                if random.random() < self.mutation_rate:
                    # Flag to indicate if a suitable mutation has been found.
                    flag = True

                    # Iterate over the family's preferences.
                    for j in range(self.preferences_num):
                        # Get the day candidate from the family's preferences
                        day_candidate = self.family_pref_matrix[gene_index][j]

                        # Calculate the current load of the candidate day.
                        day_load = sum(chromosome == day_candidate)

                        # Check if the candidate day can accommodate the family
                        # without exceeding the maximum capacity.
                        if day_load + self.family_sizes[gene_index] < self.MAX_PEOPLE:
                            # If the candidate day is suitable, assign the family to
                            # this day.
                            chromosome[gene_index] = day_candidate

                            # Set the flag to False to indicate a successful mutation.
                            flag = False

                            # Break the loop as the mutation has been applied.
                            break

                    # If no suitable day was found in the family's preferences,
                    # assign the family to a random day.
                    if not flag:
                        chromosome[gene_index] = np.random.randint(1, self.num_days + 1)
                else:
                    # If the mutation rate was not met, assign the family to a random day.
                    chromosome[gene_index] = np.random.randint(1, self.num_days + 1)

        # Return the updated chromosome.
        return chromosome

    def crossover(self, parent1: np.ndarray, parent2: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """
        Performs crossover between two parents to produce offspring.

        This method combines two parent chromosomes to create two new offspring.
        The crossover process involves selecting a random number of crossover
        points and swapping the segments between these points between the
        two parents. This introduces variability in the offspring and helps
        in exploring the solution space more thoroughly.

        Args:
            parent1 (numpy.ndarray): First parent chromosome.
            parent2 (numpy.ndarray): Second parent chromosome.

        Returns:
            tuple: Two offspring chromosomes.
        """

        # Decide whether to perform crossover based on the crossover rate.
        if random.random() < self.crossover_rate:
          # Randomly choose the number of crossover points from 1 to 3.
          # This introduces variability in the crossover process.
          num_crossover_points = random.randint(1, 3)

          # Generate a sorted list of crossover points.
          # This ensures that the crossover points are distinct and ordered.
          crossover_points = sorted(random.sample(range(1, len(parent1)), num_crossover_points))

          # Add the start and end points to the list of crossover points.
          crossover_points.insert(0, 0)
          crossover_points.append(len(parent1))

          # Initialize the offspring chromosomes.
          child1 = []
          child2 = []

          # Iterate over the crossover points to create the offspring.
          for i in range(len(crossover_points) - 1):
              # Determine which parent to use for the current segment based on the index.
              # This alternates between parents for each segment.
              if i % 2 == 0:
                  # Use the segment from the first parent.
                  segment1 = copy.deepcopy(parent1[crossover_points[i]: crossover_points[i + 1]])
                  # Use the segment from the second parent.
                  segment2 = copy.deepcopy(parent2[crossover_points[i]: crossover_points[i + 1]])
              else:
                  # Use the segment from the second parent.
                  segment1 = copy.deepcopy(parent2[crossover_points[i]: crossover_points[i + 1]])
                  # Use the segment from the first parent.
                  segment2 = copy.deepcopy(parent1[crossover_points[i]: crossover_points[i + 1]])

              # Append the segments to the corresponding offspring.
              child1.extend(segment1)
              child2.extend(segment2)

          # Convert the lists to numpy arrays for consistency with the rest of the code.
          child1 = np.array(child1)
          child2 = np.array(child2)

          # Return the offspring.
          return child1, child2
        else:
          # If the crossover rate was not met, return the parents unchanged.
          return parent1, parent2

    def select_parents(self, population: List[np.ndarray], scores: List[float], parents_size: int) -> List[np.ndarray]:
        """
        Selects parents for crossover based on their fitness scores.


        This method selects a specified number of parents from the population for crossover.
        The selection is based on the fitness scores of the chromosomes, with higher-scoring
        chromosomes having a higher chance of being selected. This ensures that the offspring
        produced by the crossover process are likely to be better solutions than the parents.

        Args:
            population (List[numpy.ndarray]): List of chromosomes (parents) from which to select.
            scores (List[float]): List of fitness scores corresponding to each chromosome in the population.
            parents_size (int): Number of parents to select.

        Returns:
            List[numpy.ndarray]: Selected parents.
        """
        # Initialize an empty list to store the selected parents.
        selected_parents = []

        # Iterate over the number of parents to select.
        for _ in range(parents_size):
            # Randomly select two indices from the population.
            idx1, idx2 = np.random.choice(range(len(population)), size=2, replace=False)

            # Compare the fitness scores of the two selected chromosomes.
            # The chromosome with the lower score is more likely to be selected,
            # as it represents a better solution.
            if scores[idx1] < scores[idx2]:
                # If the first chromosome has a lower score, add it to the selected parents.
                selected_parents.append(copy.deepcopy(population[idx1]))
            else:
                # If the second chromosome has a lower score, add it to the selected parents.
                selected_parents.append(copy.deepcopy(population[idx2]))

        # Return the list of selected parents.
        return selected_parents

    def replace(self, offsprings: List[np.ndarray], population_scores: List[float]) -> List[np.ndarray]:
        """
        Replaces the old population with the new one (including offspring).

        This method combines the current population with the newly produced offspring and selects the best
        chromosomes to form the new population. The selection is based on the fitness scores of the chromosomes,
        with higher-scoring chromosomes having a higher chance of being selected. This ensures that the new
        population is likely to be better than the old one.


        Args:
            offsprings (List[numpy.ndarray]): List of offspring chromosomes produced by
                                              crossover and mutation.
            population_scores (List[float]): List of fitness scores corresponding to
                                             each chromosome in the current population.

        Returns:
            List[numpy.ndarray]: New population after replacement.
        """
        # Combine the current population with the offsprings.
        # Each offspring is paired with its fitness score, which is calculated by applying the fitness function.
        combined_population = [(chromosome, score) for chromosome, score in zip(self.population, population_scores)] + \
                            [(offsprings[i], self.fitness_function(offsprings[i])) for i in range(len(offsprings))]

        # Sort the combined population based on the fitness scores.
        # This ensures that the chromosomes with the lowest scores are at the beginning of the list.
        combined_population.sort(key=lambda x: x[1])

        # Select the best chromosomes from the combined population to form the new population.
        # The size of the new population is the same as the original population size.
        new_population = [chromosome for chromosome, score in combined_population[:self.population_size]]

        # Return the new population.
        return new_population

    def save_solution(self):
        """
        Saves the best solution to a CSV file.

        This method retrieves the best chromosome found by the genetic algorithm and saves it to a CSV file.
        The best chromosome represents the optimal assignment of families to days, based on the fitness function.
        The solution is saved in a format that includes the family ID and the assigned day for each family.

        """
        # Retrieve the best chromosome - the chromosome with the lowest fitness score,
        # representing the best solution found.
        best_chromosome = self.best_solution

        # Initialize an empty list to store the solution.
        # Each element in the list will be a list containing the family ID and the assigned day.
        answer = []
        for family_id, day in enumerate(best_chromosome):
            # Append the family ID and the assigned day to the solution list.
            answer.append([family_id, day])

        # Convert the solution list to a pandas DataFrame.
        # The DataFrame has two columns: 'family_id' and 'assigned_day'.
        submission_df = pd.DataFrame(answer, columns=['family_id', 'assigned_day'])

        # Save the DataFrame to the specified file path
        # This file will contain the best solution found by the genetic algorithm.
        submission_df.to_csv(self.answer_location, index=False)

        # Print a message indicating that the solution was successfully saved.
        print("Solution was successfully saved to", self.answer_location)

    def run(self):
        """
        Runs the genetic algorithm for scheduling.

        This method executes the genetic algorithm for a specified number of generations. It includes the
        initialization of the population, the evaluation of fitness for each chromosome, the selection of
        parents for crossover, the crossover and mutation of offspring, and the replacement of the old
        population with the new one. The algorithm also includes a mechanism to regenerate the initial
        population if the fitness does not improve for a certain number of generations, to avoid
        premature convergence to suboptimal solutions.
        """
        log_file = open('logs.txt', 'w')
        # Iterate over the specified number of generations.
        for generation in range(self.num_generations):
            # Evaluate the fitness of each chromosome in the population
            population_scores = [self.fitness_function(chromosome) for chromosome in self.population]

            # Remember the current best fitness
            current_best_fitness = min(population_scores)
            # If the current best fitness is lower than the previous best, update the best solution and fitness.
            if current_best_fitness < self.best_fitness:
                self.best_solution = copy.deepcopy(self.population[population_scores.index(current_best_fitness)])
                self.best_fitness = current_best_fitness

            # Every 200 generations, save the current best solution.
            if generation % 200 == 0:
              self.save_solution()

            # Print the current generation number, the best fitness, the best fitness in the current population,
            # and the mean fitness of the population.
            print(f"Generation {generation + 1}: Best fitness = {self.best_fitness}")
            log_file.write(f"Generation {generation + 1}: Best fitness = {self.best_fitness}\n")

            # Select parents for crossover
            parents = self.select_parents(self.population, population_scores, self.population_size // 2)

            # Perform crossover to create offsprings
            offsprings = []
            for i in range(len(parents) - 1):
                child1, child2 = self.crossover(parents[i], parents[i + 1])
                offsprings.append(child1)
                offsprings.append(child2)

            # Apply mutation to the offspring
            for i in range(len(offsprings)):
                offsprings[i] = self.mutation(offsprings[i])

            # Replace the old population with the new one (including offspring)
            self.population = self.replace(offsprings, population_scores)



In [ ]:
data_file_path = 'family_data.csv'
answer_file_path = 'submission.csv'
scheduler = GeneticScheduler(data_file_path, answer_file_path)  # Initialize the scheduler
scheduler.run()  # Execute the genetic algorithm
